In [2]:
import pandas as pd
import numpy as np
from typing import Dict, List

In [ ]:
class ColumnPrioritizer:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        
        # Hard-coded duplicate column groups based on the Salesforce dataset
        self.duplicate_groups = {
            'account_name': ['Account Name', 'account_name', 'AccountName'],
            'contact_email': ['Contact Email', 'contact_email'],
            'created_date': ['Created Date', 'created_date'],
            'lead_source': ['Lead Source', 'lead_source'],
            'opportunity_amount': ['Opportunity Amount', 'opportunity_amount'],
            'is_active': ['Is Active', 'is_active'],
            'sfdc_id': ['SFDC ID', 'sfdc_id'],
            'annual_revenue': ['Annual Revenue', 'annual_revenue']
        }
    
    def completeness_score(self, column_name: str) -> float:
        ## Calculate completeness score for a column (0-1).
        if column_name not in self.df.columns:
            return 0.0
        return self.df[column_name].notna().sum() / len(self.df)
    
    def quality_score(self, column_name: str) -> float:
        ## Calculate quality score for a column (0-1).
        if column_name not in self.df.columns:
            return 0.0
        
        series = self.df[column_name].dropna()
        if len(series) == 0:
            return 0.0
        
        # Check for error indicators
        error_indicators = ['not_a_date', 'noemail', 'invalid@', 'BAD_ID', 'N/A', 'NaT']
        error_count = sum(1 for value in series.astype(str) 
                         if any(indicator in str(value) for indicator in error_indicators))
        
        return max(0.0, 1.0 - (error_count / len(series)))
    